In [1]:
import csv
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

# rotatation matrix to convert from body to world frame
def Rmat(phi, theta, psi):
    Rx = np.array([[1, 0, 0], [0, np.cos(phi), -np.sin(phi)], [0, np.sin(phi), np.cos(phi)]])
    Ry = np.array([[np.cos(theta), 0, np.sin(theta)],[0, 1, 0],[-np.sin(theta), 0, np.cos(theta)]])
    Rz = np.array([[np.cos(psi), -np.sin(psi), 0],[np.sin(psi), np.cos(psi), 0], [0, 0, 1]])
    R = Rz@Ry@Rx
    return R

# quaternion functions https://personal.utdallas.edu/~sxb027100/dock/quaternion.html
def quadMult(ql, qr):
    qli, qlx, qly, qlz = ql
    qri, qrx, qry, qrz = qr
    res = np.array([
        qli*qri - qlx*qrx - qly*qry - qlz*qrz,
        qlx*qri + qli*qrx + qly*qrz - qlz*qry,
        qli*qry - qlx*qrz + qly*qri + qlz*qrx,
        qli*qrz + qlx*qry - qly*qrx + qlz*qri
    ])
    return res

def quadRotate(q, v):
    qi, qx, qy, qz = q
    vx, vy, vz = v
    qv = np.array([0, vx, vy, vz])
    q_qv = quadMult(q, qv)
    qv = quadMult(q_qv, np.array([qi, -qx, -qy, -qz]))
    return qv[1:]

def quat_of_axang(ax, ang):
    ang2 = ang/2
    cang2 = np.cos(ang2)
    sang2 = np.sin(ang2)
    q = np.array([
        cang2,
        ax[0]*sang2,
        ax[1]*sang2,
        ax[2]*sang2
    ])
    return q

# testing new att setpoint calculation
def att_thrust_sp_from_acc(quat_NED, acc_sp_NED):
    # we need to rotate the drone such that the z-up axis is aligned with accSpNed-g
    # let v1 = accSpNed-g, v2 = z-up (both in NED frame)
    # then we need to rotate v2 to v1
    # the rotation axis is the cross product of v1 and v2
    # the rotation angle is the angle between v1 and v2
    
    v1 = acc_sp_NED - np.array([0, 0, 9.81])
    v2_body = np.array([0, 0, -1])
    # get v2 in NED frame (rotate v2_body by quat_NED)
    v2 = quadRotate(quat_NED, v2_body)
    # rotation axis
    axis = np.cross(v2, v1)
    axis = axis/np.linalg.norm(axis)
    angle = np.arccos(np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))
    # convert axis and angle to quaternion
    attErrNed = quat_of_axang(axis, angle)
    # attitude setpoint
    att_sp_NED = quadMult(attErrNed, quat_NED)
    
    # thrust setpoint (acc_sp projected onto z-up axis)
    T_sp = -np.dot(v1, v2)
    
    # thrust setpoint 'direct'
    T_sp = -np.linalg.norm(v1)
    
    # EXTRA: get rate sp
    # we get the axis in body frame:
    quat_NED_inv = np.array([quat_NED[0], -quat_NED[1], -quat_NED[2], -quat_NED[3]])
    axis_body = quadRotate(quat_NED_inv, axis)
    
    # and scale the axis by 'angle' to get the att_error_axis
    att_error_axis = axis_body*angle
    
    # multiply by a gain to get the rate setpoint
    rate_sp = np.array([10, 10, 5]) * att_error_axis    
    
    return att_sp_NED, T_sp, rate_sp


def quat_to_euler(q):
    qw, qx, qy, qz = q
    phi = np.arctan2(2*(qw*qx + qy*qz), 1 - 2*(qx**2 + qy**2))
    theta = np.arcsin(2*(qw*qy - qz*qx))
    psi = np.arctan2(2*(qw*qz + qx*qy), 1 - 2*(qy**2 + qz**2))
    return phi, theta, psi
    

def load_flight_data(file_name):
    with open(file_name) as file:
        reader = csv.reader(file)
        data = list(reader)
        
        # separate header and data
        header = [d for d in data if len(d)==2]
        header = {d[0]: d[1] for d in header}
        
        data = [d for d in data if len(d)>2]
        keys = data[0]
        data = data[1:]
        data = np.array([[float(d) if d else np.nan for d in row] for row in data]) # set empty strings to nan
        data = dict(zip(keys,data.T))
        
        # renaming keys
        data['t'] = data.pop('time')*1e-6 # us to s
        data['t'] -= data['t'][0] # start at 0

        # STATE ESTIMATE
        data['x'] = data.pop('pos[0]')*1e-3 # mm to m
        data['y'] = data.pop('pos[1]')*1e-3 # mm to m
        data['z'] = data.pop('pos[2]')*1e-3 # mm to m
        
        data['vx'] = data.pop('vel[0]')*1e-2 # cm/s to m/s
        data['vy'] = data.pop('vel[1]')*1e-2 # cm/s to m/s
        data['vz'] = data.pop('vel[2]')*1e-2 # cm/s to m/s
        
        quat_scaling = ((127 << 6) - 1) # float from -1 to +1 to VB that takes up 2 bytes max
        data['qw'] = data.pop('quat[0]')/quat_scaling
        data['qx'] = data.pop('quat[1]')/quat_scaling
        data['qy'] = data.pop('quat[2]')/quat_scaling
        data['qz'] = data.pop('quat[3]')/quat_scaling
        
        # quat to eulers
        data['phi'] = np.arctan2(2*(data['qw']*data['qx'] + data['qy']*data['qz']), 1 - 2*(data['qx']**2 + data['qy']**2))
        data['theta'] = np.arcsin(2*(data['qw']*data['qy'] - data['qz']*data['qx']))
        data['psi'] = np.arctan2(2*(data['qw']*data['qz'] + data['qx']*data['qy']), 1 - 2*(data['qy']**2 + data['qz']**2))
        
        # SETPOINTS
        data['x_sp'] = data.pop('posSp[0]')*1e-3 # mm to m
        data['y_sp'] = data.pop('posSp[1]')*1e-3
        data['z_sp'] = data.pop('posSp[2]')*1e-3 
        
        data['vx_sp'] = data.pop('velSp[0]')*1e-2 # cm/s to m/s
        data['vy_sp'] = data.pop('velSp[1]')*1e-2
        data['vz_sp'] = data.pop('velSp[2]')*1e-2
        
        data['awx_sp'] = data.pop('accSp[0]')*1e-2 # cm/s^2 to m/s^2
        data['awy_sp'] = data.pop('accSp[1]')*1e-2
        data['awz_sp'] = data.pop('accSp[2]')*1e-2
        
        data['qw_sp'] = data.pop('quatSp[0]')/quat_scaling
        data['qx_sp'] = data.pop('quatSp[1]')/quat_scaling
        data['qy_sp'] = data.pop('quatSp[2]')/quat_scaling
        data['qz_sp'] = data.pop('quatSp[3]')/quat_scaling
        
        # quat to eulers
        data['phi_sp'] = np.arctan2(2*(data['qw_sp']*data['qx_sp'] + data['qy_sp']*data['qz_sp']), 1 - 2*(data['qx_sp']**2 + data['qy_sp']**2))
        data['theta_sp'] = np.arcsin(2*(data['qw_sp']*data['qy_sp'] - data['qz_sp']*data['qx_sp']))
        data['psi_sp'] = np.arctan2(2*(data['qw_sp']*data['qz_sp'] + data['qx_sp']*data['qy_sp']), 1 - 2*(data['qy_sp']**2 + data['qz_sp']**2))
        
        # euler_sp = np.array([
        #     quat_to_euler([qw, qx, qy, qz]) for qw, qx, qy, qz in zip(data['qw_sp'], data['qx_sp'], data['qy_sp'], data['qz_sp'])
        # ])
        # data['phi_sp2'] = euler_sp[:, 0]
        # data['theta_sp2'] = euler_sp[:, 1]
        # data['psi_sp2'] = euler_sp[:, 2] 
        
        data['p_sp'] = data.pop('gyroSp[0]')*np.pi/180 # deg/s to rad/s
        data['q_sp'] = data.pop('gyroSp[1]')*np.pi/180
        data['r_sp'] = data.pop('gyroSp[2]')*np.pi/180
        
        # data['dp_sp'] = data.pop('alphaSp[0]')*np.pi/180 # deg/s^2 to rad/s^2
        # data['dq_sp'] = data.pop('alphaSp[1]')*np.pi/180
        # data['dr_sp'] = data.pop('alphaSp[2]')*np.pi/180
        
        data['spf_sp_x'] = data.pop('spfSp[0]')/100
        data['spf_sp_y'] = data.pop('spfSp[1]')/100
        data['spf_sp_z'] = data.pop('spfSp[2]')/100
        
        data['T_sp'] = data['spf_sp_z']
        
        # INDI
        data['alpha[0]'] = data.pop('alpha[0]')*10*np.pi/180
        data['alpha[1]'] = data.pop('alpha[1]')*10*np.pi/180
        data['alpha[2]'] = data.pop('alpha[2]')*10*np.pi/180
        
        data['omega[0]'] = data.pop('omega[0]') # already in rad/s
        data['omega[1]'] = data.pop('omega[1]')
        data['omega[2]'] = data.pop('omega[2]')
        data['omega[3]'] = data.pop('omega[3]')
        
        data['omega_dot[0]'] = data.pop('omega_dot[0]')*100
        data['omega_dot[1]'] = data.pop('omega_dot[1]')*100
        data['omega_dot[2]'] = data.pop('omega_dot[2]')*100
        data['omega_dot[3]'] = data.pop('omega_dot[3]')*100
        
        data['u[0]'] = data['u[0]']/quat_scaling
        data['u[1]'] = data['u[1]']/quat_scaling
        data['u[2]'] = data['u[2]']/quat_scaling
        data['u[3]'] = data['u[2]']/quat_scaling
        
        
        # MOTOR CMDS
        motor_limits = header['motorOutput'].split(',')
        umin = int(motor_limits[0])
        umax = int(motor_limits[1])
        data['motor_min'] = umin
        data['motor_max'] = umax
        data['u1'] = (data['motor[0]'] - umin)/(umax - umin)
        data['u2'] = (data['motor[1]'] - umin)/(umax - umin)
        data['u3'] = (data['motor[2]'] - umin)/(umax - umin)
        data['u4'] = (data['motor[3]'] - umin)/(umax - umin)
        
        # OPTITRACK
        data['x_opti'] = data.pop('extPos[0]')*1e-3 # mm to m
        data['y_opti'] = data.pop('extPos[1]')*1e-3 # mm to m
        data['z_opti'] = data.pop('extPos[2]')*1e-3 # mm to m
        
        data['vx_opti'] = data.pop('extVel[0]')*1e-2 # cm/s to m/s
        data['vy_opti'] = data.pop('extVel[1]')*1e-2 # cm/s to m/s
        data['vz_opti'] = data.pop('extVel[2]')*1e-2 # cm/s to m/s
        
        data['phi_opti'] = data.pop('extAtt[0]')/1000 # mrad to rad
        data['theta_opti'] = data.pop('extAtt[1]')/1000 # mrad to rad
        data['psi_opti'] = data.pop('extAtt[2]')/1000 # mrad to rad
        
        data['vel_test'] = np.sqrt(data['vx_opti']**2 + data['vy_opti']**2 + data['vz_opti']**2)
        
        # OPTITRACK INTERPOLATED
        updates = (np.diff(data['x_opti']) != 0) | (np.diff(data['y_opti']) != 0) | (np.diff(data['z_opti']) != 0) | (np.diff(data['phi_opti']) != 0) | (np.diff(data['theta_opti']) != 0) | (np.diff(data['psi_opti']) != 0)
        updates = np.where(updates)[0]+1
        
        # data['t_opti_rec'] = data['t'][updates]
        # data['t_opti_sent'] = data['extTime'][updates]/1000 # ms to s
        # data['x_opti_int'] = np.interp(data['t'], data['t'][updates], data['x_opti'][updates])
        # data['y_opti_int'] = np.interp(data['t'], data['t'][updates], data['y_opti'][updates])
        # data['z_opti_int'] = np.interp(data['t'], data['t'][updates], data['z_opti'][updates])
        # data['phi_opti_int'] = np.interp(data['t'], data['t'][updates], data['phi_opti'][updates])
        # data['theta_opti_int'] = np.interp(data['t'], data['t'][updates], data['theta_opti'][updates])
        # data['psi_opti_int'] = np.interp(data['t'], data['t'][updates], data['psi_opti'][updates])
        
        
        # IMU
        gyro_scale = np.pi/180 # deg/s to rad/s
        if bool(float(header['blackbox_high_resolution'])):
            gyro_scale /= 10
        data['p'] = data.pop('gyroADC[0]')*gyro_scale       # (from FLU to FRD)
        data['q'] =-data.pop('gyroADC[1]')*gyro_scale
        data['r'] =-data.pop('gyroADC[2]')*gyro_scale
        
        acc_scale = 9.81/float(header['acc_1G'])
        data['ax'] = data.pop('accSmooth[0]')*acc_scale     # (from FLU to FRD)
        data['ay'] =-data.pop('accSmooth[1]')*acc_scale
        data['az'] =-data.pop('accSmooth[2]')*acc_scale
        
        if 'accUnfiltered[0]' in data.keys():
            data['ax_unfiltered'] = data.pop('accUnfiltered[0]')*acc_scale     # (from FLU to FRD)
            data['ay_unfiltered'] =-data.pop('accUnfiltered[1]')*acc_scale
            data['az_unfiltered'] =-data.pop('accUnfiltered[2]')*acc_scale
        
        # filter acc
        cutoff = 8 # Hz
        sos = sp.signal.butter(2, cutoff, 'low', fs=1/np.mean(np.diff(data['t'])), output='sos')
        data['ax_filt'] = sp.signal.sosfiltfilt(sos, data['ax'])
        data['ay_filt'] = sp.signal.sosfiltfilt(sos, data['ay'])
        data['az_filt'] = sp.signal.sosfiltfilt(sos, data['az'])
        
        # EKF
        if 'ekf_pos[0]' in data:
            data['ekf_x'] = data.pop('ekf_pos[0]')*1e-3 # mm to m
            data['ekf_y'] = data.pop('ekf_pos[1]')*1e-3 # mm to m
            data['ekf_z'] = data.pop('ekf_pos[2]')*1e-3 # mm to m
            data['ekf_vx'] = data.pop('ekf_vel[0]')*1e-2 # cm/s to m/s
            data['ekf_vy'] = data.pop('ekf_vel[1]')*1e-2 # cm/s to m/s
            data['ekf_vz'] = data.pop('ekf_vel[2]')*1e-2 # cm/s to m/s
            data['ekf_phi'] = data.pop('ekf_att[0]')/1000 # mrad to rad
            data['ekf_theta'] = data.pop('ekf_att[1]')/1000 # mrad to rad
            data['ekf_psi'] = data.pop('ekf_att[2]')/1000 # mrad to rad
            data['ekf_acc_b_x'] = data.pop('ekf_acc_b[0]')/1000 # mm/s^2 to m/s^2
            data['ekf_acc_b_y'] = data.pop('ekf_acc_b[1]')/1000 # mm/s^2 to m/s^2
            data['ekf_acc_b_z'] = data.pop('ekf_acc_b[2]')/1000 # mm/s^2 to m/s^2
            data['ekf_gyro_b_x'] = data.pop('ekf_gyro_b[0]')*np.pi/180
            data['ekf_gyro_b_y'] = data.pop('ekf_gyro_b[1]')*np.pi/180
            data['ekf_gyro_b_z'] = data.pop('ekf_gyro_b[2]')*np.pi/180
        
            data['ax_filt_unbiased'] = data['ax_filt'] - data['ekf_acc_b_x']
            data['ay_filt_unbiased'] = data['ay_filt'] - data['ekf_acc_b_y']
            data['az_filt_unbiased'] = data['az_filt'] - data['ekf_acc_b_z']
    
        # VIO
        if 'vioPos[0]' in data and False:
            print('vioooo')
            print(np.any(data['vioPos[0]'] != 0))
            data['x_vio'] = data.pop('vioPos[0]')*1e-3 # mm to m
            data['y_vio'] = data.pop('vioPos[1]')*1e-3
            data['z_vio'] = data.pop('vioPos[2]')*1e-3
            data['vx_vio'] = data.pop('vioVel[0]')*1e-2 # cm/s to m/s
            data['vy_vio'] = data.pop('vioVel[1]')*1e-2
            data['vz_vio'] = data.pop('vioVel[2]')*1e-2
            data['qw_vio'] = data.pop('vioQuat[0]')/quat_scaling
            data['qx_vio'] = data.pop('vioQuat[1]')/quat_scaling
            data['qy_vio'] = data.pop('vioQuat[2]')/quat_scaling
            data['qz_vio'] = data.pop('vioQuat[3]')/quat_scaling
            data['phi_vio'] = np.arctan2(2*(data['qw_vio']*data['qx_vio'] + data['qy_vio']*data['qz_vio']), 1 - 2*(data['qx_vio']**2 + data['qy_vio']**2))
            data['theta_vio'] = np.arcsin(2*(data['qw_vio']*data['qy_vio'] - data['qz_vio']*data['qx_vio']))
            data['psi_vio'] = np.arctan2(2*(data['qw_vio']*data['qz_vio'] + data['qx_vio']*data['qy_vio']), 1 - 2*(data['qy_vio']**2 + data['qz_vio']**2))
            data['p_vio'] = data.pop('vioRate[0]')*np.pi/180 # deg/s to rad/s
            data['q_vio'] = data.pop('vioRate[1]')*np.pi/180
            data['r_vio'] = data.pop('vioRate[2]')*np.pi/180
    
        # EXTRA
        data['v'] = np.sqrt(data['vx']**2 + data['vy']**2 + data['vz']**2)

        # body velocities
        v_body = np.stack([
            Rmat(phi, theta, psi).T@[vx, vy, vz]
            for vx, vy, vz, phi, theta, psi
            in zip(data['vx'],data['vy'],data['vz'],data['phi'],data['theta'],data['psi'])
        ])
        data['vbx'] = v_body[:,0]
        data['vby'] = v_body[:,1]
        data['vbz'] = v_body[:,2]
        
        # simple drag model
        data['Dx'] = -0.43291866*data['vbx']
        data['Dy'] = -0.49557959*data['vby']
        
        # world accelerations
        a_world = np.stack([
            Rmat(phi, theta, psi)@np.array([ax, ay, az]) + np.array([0, 0, 9.81])
            for ax, ay, az, phi, theta, psi
            in zip(data['ax'],data['ay'],data['az'],data['phi'],data['theta'],data['psi'])
        ])
        data['awx'] = a_world[:,0]
        data['awy'] = a_world[:,1]
        data['awz'] = a_world[:,2]
        
        data['awx_filt'] = sp.signal.sosfiltfilt(sos, data['awx'])
        data['awy_filt'] = sp.signal.sosfiltfilt(sos, data['awy'])
        data['awz_filt'] = sp.signal.sosfiltfilt(sos, data['awz'])
        
        # reconstruct acc setpoint by rotating the thrust setpoint with the attitude setpoint
        a_world_sp_rec = np.stack([
            Rmat(phi, theta, psi)@np.array([0,0,T]) + np.array([0, 0, 9.81])
            for T, phi, theta, psi
            in zip(data['T_sp'],data['phi_sp'],data['theta_sp'],data['psi_sp'])
        ])
        data['awx_sp_rec'] = a_world_sp_rec[:, 0]
        data['awy_sp_rec'] = a_world_sp_rec[:, 1]
        data['awz_sp_rec'] = a_world_sp_rec[:, 2]
        
        # test att, thrust setpoint calculation
        att_sp_test = np.array([
            att_thrust_sp_from_acc(np.array([qw,qx,qy,qz]), np.array([awx,awy,awz]))[0] for 
            qw,qx,qy,qz,awx,awy,awz in zip(data['qw'],data['qx'],data['qy'],data['qz'],data['awx_sp'],data['awy_sp'],data['awz_sp'])
        ])
        
        data['qw_sp_test'] = att_sp_test[:, 0]
        data['qx_sp_test'] = att_sp_test[:, 1]
        data['qy_sp_test'] = att_sp_test[:, 2]
        data['qz_sp_test'] = att_sp_test[:, 3]
        
        data['phi_sp_test'] = np.arctan2(2*(data['qw_sp_test']*data['qx_sp_test'] + data['qy_sp_test']*data['qz_sp_test']), 1 - 2*(data['qx_sp_test']**2 + data['qy_sp_test']**2))
        data['theta_sp_test'] = np.arcsin(2*(data['qw_sp_test']*data['qy_sp_test'] - data['qz_sp_test']*data['qx_sp_test']))
        data['psi_sp_test'] = np.arctan2(2*(data['qw_sp_test']*data['qz_sp_test'] + data['qx_sp_test']*data['qy_sp_test']), 1 - 2*(data['qy_sp_test']**2 + data['qz_sp_test']**2))
        
        # Thrust setpoint
        thrust_sp_test = np.array([
            att_thrust_sp_from_acc(np.array([qw,qx,qy,qz]), np.array([awx,awy,awz]))[1] for 
            qw,qx,qy,qz,awx,awy,awz in zip(data['qw'],data['qx'],data['qy'],data['qz'],data['awx_sp'],data['awy_sp'],data['awz_sp'])
        ])
        data['T_sp_test'] = thrust_sp_test
        
        # Rate setpoint
        rate_sp_test = np.array([
            att_thrust_sp_from_acc(np.array([qw,qx,qy,qz]), np.array([awx,awy,awz]))[2] for 
            qw,qx,qy,qz,awx,awy,awz in zip(data['qw'],data['qx'],data['qy'],data['qz'],data['awx_sp'],data['awy_sp'],data['awz_sp'])
        ])
        data['p_sp_test'] = rate_sp_test[:, 0]
        data['q_sp_test'] = rate_sp_test[:, 1]
        data['r_sp_test'] = rate_sp_test[:, 2]
        
        # reconstruct acc setpoint from test att, thrust setpoint
        a_world_sp_rec_test = np.stack([
            Rmat(phi, theta, psi)@np.array([0,0,T]) + np.array([0, 0, 9.81])
            for T, phi, theta, psi
            in zip(data['T_sp_test'],data['phi_sp_test'],data['theta_sp_test'],data['psi_sp_test'])
        ])
        data['awx_sp_rec_test'] = a_world_sp_rec_test[:, 0]
        data['awy_sp_rec_test'] = a_world_sp_rec_test[:, 1]
        data['awz_sp_rec_test'] = a_world_sp_rec_test[:, 2]
        
        return data

In [2]:
from quadcopter_animation import animation
import importlib
importlib.reload(animation)

# RACE TRACK:
r=3.
num = 8
gate_pos = np.array([
    [r*np.cos(angle), r*np.sin(angle), -1.5] for angle in np.linspace(0,2*np.pi,num,endpoint=False)
])
gate_yaw = np.array([np.pi/2 + i*2*np.pi/num for i in range(num)])

def animate_data(data):
    animation.animate(
        data['t'],
        data['x'], data['y'], data['z'],
        data['phi'], data['theta'], data['psi'],
        np.stack([data['u1'], data['u2'], data['u3'], data['u4']], axis=1),
        target=np.stack([data['x_sp'], data['y_sp'], data['z_sp']], axis=1)
    )
    
def animate_data_double(data1, data2):
    animation.animate(
        [data1['t'], data2['t']],
        [data1['x'], data2['x']],
        [data1['y'], data2['y']],
        [data1['z'], data2['z']],
        [data1['phi'], data2['phi']],
        [data1['theta'], data2['theta']],
        [data1['psi'], data2['psi']],
        [
            np.stack([data1['u1'], data1['u2'], data1['u3'], data1['u4']], axis=1),
            np.stack([data2['u1'], data2['u2'], data2['u3'], data2['u4']], axis=1)
        ],
        multiple_trajectories=True,
        simultaneous=True,
        colors=[(255,0,0), (0,255,0)]
    )
    
def animate_data_multiple(*data_list):
    # color map
    import matplotlib.cm as cm
    colors_ = cm.get_cmap('jet', len(data_list))
    # to int
    colors = lambda i: tuple(int(c*255) for c in colors_(i)[:-1])
    
    animation.animate(
        [d['t'] for d in data_list],
        [d['x'] for d in data_list],
        [d['y'] for d in data_list],
        [d['z'] for d in data_list],
        [d['phi'] for d in data_list],
        [d['theta'] for d in data_list],
        [d['psi'] for d in data_list],
        [np.stack([d['u1'], d['u2'], d['u3'], d['u4']], axis=1) for d in data_list],
        # target=[np.stack([d['x_opti'], d['y_opti'], d['z_opti']], axis=1) for d in data_list],
        multiple_trajectories=True,
        simultaneous=True,
        colors=[colors(i) for i in range(len(data_list))],
        gate_pos=gate_pos,
        gate_yaw=gate_yaw
    )
    
def animate_data_multiple2(*data_list, **kwargs):
    # color map
    import matplotlib.cm as cm
    colors_ = cm.get_cmap('jet', len(data_list))
    # to int
    colors = lambda i: tuple(int(c*255) for c in colors_(i)[:-1])
    colors2 = lambda i: tuple(int(c*55+200) for c in colors_(i)[:-1])
    
    animation.animate(
        [d['t'] for d in data_list]*2,
        [d['x'] for d in data_list] + [d['x_vio'] for d in data_list],
        [d['y'] for d in data_list] + [d['y_vio'] for d in data_list],
        [d['z'] for d in data_list] + [d['z_vio'] for d in data_list],
        [d['phi'] for d in data_list] + [d['phi_vio'] for d in data_list],
        [d['theta'] for d in data_list] + [d['theta_vio'] for d in data_list],
        [d['psi'] for d in data_list] + [d['psi_vio'] for d in data_list],
        [np.stack([d['u1'], d['u2'], d['u3'], d['u4']], axis=1) for d in data_list]+[np.zeros((len(data_list[0]['t']), 4)) for _ in data_list],
        target=np.stack([data_list[0]['x_sp'], data_list[0]['y_sp'], data_list[0]['z_sp']], axis=1),
        multiple_trajectories=True,
        simultaneous=True,
        colors=[colors(i) for i in range(len(data_list))]+[colors2(i) for i in range(len(data_list))],
        gate_pos=gate_pos,
        gate_yaw=gate_yaw
    )
    
    
def animate_data2(data):
    animation.animate(
        [data['t'], data['t']],
        [data['x'], data['x_opti']],
        [data['y'], data['y_opti']],
        [data['z'], data['z_opti']],
        [data['phi'], data['phi_opti']],
        [data['theta'], data['theta_opti']],
        [data['psi'], data['psi_opti']],
        [
            np.stack([data['u1'], data['u2'], data['u3'], data['u4']], axis=1),
            np.stack([data['u1'], data['u2'], data['u3'], data['u4']], axis=1)
        ],
        multiple_trajectories=True,
        simultaneous=True,
        target=np.stack([data['x_sp'], data['y_sp'], data['z_sp']], axis=1),
        colors=[(255,0,0), (0,255,0)]
    )

In [3]:
path2 = 'flight_data/26July_erins_newnew_damping_nn_circle.csv'
path1 = 'flight_data/15July_new_drone_oring_damping.csv'
# path = 'flight_data/29July_vio_desk_test2.csv'

data1 = load_flight_data(path1)
data2 = load_flight_data(path2)

# return trimmed data where network is active
def get_net_activation_index(data):    
    for i in range(len(data['t'])):
        # drone in air
        if data['z'][i] < -0.5:
            # if the next m control inputs are the same, we assume the network is active
            m = 4
            if all(data['u1'][i] == data['u1'][i+j] for j in range(1,m+1)):
                return i
    return -1

def trim_net_active(data):
    idx = get_net_activation_index(data)
    data_new = {}
    for key in data:
        # if data['key'] is an array we cut it
        if isinstance(data[key], np.ndarray):
            data_new[key] = data[key][idx:].copy()
    data_new['t'] -= data_new['t'][0]
    return data_new

# data = trim_net_active(data)

data1 = trim_net_active(data1)
data2 = trim_net_active(data2)

/tmp/ipykernel_148132/2029221026.py:131: RuntimeWarning: invalid value encountered in arcsin
  data['theta'] = np.arcsin(2*(data['qw']*data['qy'] - data['qz']*data['qx']))
/tmp/ipykernel_148132/2029221026.py:154: RuntimeWarning: invalid value encountered in arcsin
  data['theta_sp'] = np.arcsin(2*(data['qw_sp']*data['qy_sp'] - data['qz_sp']*data['qx_sp']))
/tmp/ipykernel_148132/2029221026.py:360: RuntimeWarning: invalid value encountered in arcsin
  data['theta_sp_test'] = np.arcsin(2*(data['qw_sp_test']*data['qy_sp_test'] - data['qz_sp_test']*data['qx_sp_test']))


In [4]:
importlib.reload(animation)

# cut all data after 6.6s
data1_cut = {}
for key in data1:
    if isinstance(data1[key], np.ndarray):
        data1_cut[key] = data1[key][data1['t'] < 6.6].copy()
    else:
        data1_cut[key] = data1[key]

animate_data_multiple(data1_cut)

/tmp/ipykernel_148132/777011278.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors_ = cm.get_cmap('jet', len(data_list))


In [5]:
# # we make a csv with
# # x opti, y_opti, z_opti, phi_opti, theta_opti, psi_opti, vx, vy, vz
# file_name = 'realsense_drone_flight.csv'

# with open(file_name, 'w') as file:
#     writer = csv.writer(file)
#     writer.writerow([
#         't', 'x', 'y', 'z', 'phi', 'theta', 'psi', 'vx', 'vy', 'vz',
#         'x_opti', 'y_opti', 'z_opti', 'phi_opti', 'theta_opti', 'psi_opti', 'vx_opti', 'vy_opti', 'vz_opti'
#     ])
#     for t, x, y, z, phi, theta, psi, vx, vy, vz, x_opti, y_opti, z_opti, phi_opti, theta_opti, psi_opti, vx_opti, vy_opti, vz_opti in zip(
#         data['t'], -data['x'], data['y'], data['z'], data['phi'], data['theta'], data['psi'], data['vx'], data['vy'], data['vz'],
#         -data['x_opti'], data['y_opti'], data['z_opti'], data['phi_opti'], data['theta_opti'], data['psi_opti'], data['vx_opti'], data['vy_opti'], data['vz_opti']
#     ):
#         writer.writerow([
#             t, x, y, z, phi, theta, psi, vx, vy, vz,
#             x_opti, y_opti, z_opti, phi_opti, theta_opti, psi_opti, vx_opti, vy_opti, vz_opti
#         ])

In [31]:
plt.plot(data['t'], data['vbatLatest'])
# plt.plot(data['t'], data['amperageLatest'])

In [28]:
%matplotlib
data = data1
# Plot data['v'] with a horizontal line at 15.0
plt.plot(data['t'], data['v'])
data = data2
plt.plot(data['t'], data['v'])
# plt.plot(data['t'], data['amperageLatest'])
plt.ylim([0, 15])
plt.axhline(y=15.0, color='r', linestyle='-')
plt.grid()
plt.show()

Using matplotlib backend: TkAgg


In [16]:
data = data1


In [14]:
%matplotlib
# plot accelerometer + drag, thrust model
k_x = -6.0e-5
k_y = -6.0e-5
k_w = -2.5e-6

T = k_w*(data['omega[0]']**2+data['omega[1]']**2+data['omega[2]']**2+data['omega[3]']**2)
D_x = k_x*data['vbx']*(data['omega[0]']+data['omega[1]']+data['omega[2]']+data['omega[3]'])
D_y = k_y*data['vby']*(data['omega[0]']+data['omega[1]']+data['omega[2]']+data['omega[3]'])

# 3x1 subplots
fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharex=True, sharey=False)
# ax
plt.sca(axs[0])
if 'ax_unfiltered' in data.keys():
    plt.plot(data['t'], data['ax_unfiltered'], label='ax raw')
plt.plot(data['t'], data['ax'], label='ax')
plt.plot(data['t'], D_x, label='D_x')
plt.ylim([-160, 160])
plt.legend()
plt.grid()
# ay
plt.sca(axs[1])
if 'ay_unfiltered' in data.keys():
    plt.plot(data['t'], data['ay_unfiltered'], label='ay raw')
plt.plot(data['t'], data['ay'], label='ay')
plt.plot(data['t'], D_y, label='D_y')
plt.ylim([-160, 160])
plt.legend()
plt.grid()
# az
plt.sca(axs[2])
if 'az_unfiltered' in data.keys():
    plt.plot(data['t'], data['az_unfiltered'], label='az raw')
plt.plot(data['t'], data['az'], label='az')
plt.plot(data['t'], T, label='T')
plt.ylim([-160, 160])
plt.legend()
plt.grid()
plt.show()
# u1
plt.sca(axs[3])
plt.plot(data['t'], data['omega[0]'], label='w1')
plt.legend()
plt.grid()
# u2
plt.sca(axs[4])
plt.plot(data['t'], data['omega[1]'], label='w2')
plt.legend()
plt.grid()
# u3
plt.sca(axs[5])
plt.plot(data['t'], data['omega[2]'], label='w3')
plt.legend()
plt.grid()
# u4
plt.sca(axs[6])
plt.plot(data['t'], data['omega[3]'], label='w4')
plt.legend()
plt.grid()



Using matplotlib backend: TkAgg


In [30]:
# power spectral density of the accelerometer in x, y, z

# 3x1 subplots
fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharex=True, sharey=False)

# ax
plt.sca(axs[0])
f, Pxx_den = sp.signal.welch(data['ax'], fs=1/np.mean(np.diff(data['t'])), nperseg=1024)
plt.semilogy(f, Pxx_den)
plt.ylim([1e-7, 1e1])
plt.grid()
# ay
plt.sca(axs[1])
f, Pxx_den = sp.signal.welch(data['ay'], fs=1/np.mean(np.diff(data['t'])), nperseg=1024)
plt.semilogy(f, Pxx_den)
plt.ylim([1e-7, 1e1])
plt.grid()
# az
plt.sca(axs[2])
f, Pxx_den = sp.signal.welch(data['az'], fs=1/np.mean(np.diff(data['t'])), nperseg=1024)
plt.semilogy(f, Pxx_den)
plt.ylim([1e-7, 1e1])
plt.grid()
plt.show()



In [36]:
# plot u1,u2,u3,u4
# figure
fig, axs = plt.subplots(4, 1, figsize=(10, 10), sharex=True, sharey=False)
for i, u in enumerate(['u1', 'u2', 'u3', 'u4']):
    plt.sca(axs[i])
    plt.plot(data['t'], data[u], label=u)
    plt.legend()
    plt.grid()
plt.show()

## EKF plots

In [31]:
%matplotlib
ekf_updates = (np.diff(data['x_opti']) != 0) | (np.diff(data['y_opti']) != 0) | (np.diff(data['z_opti']) != 0) | (np.diff(data['phi_opti']) != 0) | (np.diff(data['theta_opti']) != 0) | (np.diff(data['psi_opti']) != 0)
ekf_updates = np.where(ekf_updates)[0]+1

# subplots 1x3 with ekf_x, ekf_y, ekf_z
fig, axs = plt.subplots(5, 3, figsize=(5,5), sharex=True, sharey='row', tight_layout=True)

# POSITION
plt.sca(axs[0,0])
plt.plot(data['t'], data['ekf_x'], label='ekf')
plt.plot(data['t'][ekf_updates], data['x_opti'][ekf_updates], label='opti')
plt.xlabel('t [s]')
plt.ylabel('x [m]')
plt.legend()
plt.sca(axs[0,1])
plt.plot(data['t'], data['ekf_y'], label='ekf')
plt.plot(data['t'][ekf_updates], data['y_opti'][ekf_updates], label='opti')
plt.xlabel('t [s]')
plt.ylabel('y [m]')
plt.legend()
plt.sca(axs[0,2])
plt.plot(data['t'], data['ekf_z'], label='ekf')
plt.plot(data['t'][ekf_updates], data['z_opti'][ekf_updates], label='opti')
plt.xlabel('t [s]')
plt.ylabel('z [m]')
plt.legend()

# VELOCITY
plt.sca(axs[1,0])
plt.plot(data['t'], data['ekf_vx'], label='ekf')
plt.xlabel('t [s]')
plt.ylabel('vx [m/s]')
plt.legend()
plt.sca(axs[1,1])
plt.plot(data['t'], data['ekf_vy'], label='ekf')
plt.xlabel('t [s]')
plt.ylabel('vy [m/s]')
plt.legend()
plt.sca(axs[1,2])
plt.plot(data['t'], data['ekf_vz'], label='ekf')
plt.xlabel('t [s]')
plt.ylabel('vz [m/s]')
plt.legend()

# ATTITUDE
plt.sca(axs[2,0])
plt.plot(data['t'], data['ekf_phi'], label='ekf')
plt.plot(data['t'][ekf_updates], data['phi_opti'][ekf_updates], label='opti')
plt.xlabel('t [s]')
plt.ylabel('phi [rad]')
plt.legend()
plt.sca(axs[2,1])
plt.plot(data['t'], data['ekf_theta'], label='ekf')
plt.plot(data['t'][ekf_updates], data['theta_opti'][ekf_updates], label='opti')
plt.xlabel('t [s]')
plt.ylabel('theta [rad]')
plt.legend()
plt.sca(axs[2,2])
ekf_psi = ((data['ekf_psi']+np.pi)%(2*np.pi))-np.pi
plt.plot(data['t'], ekf_psi, label='ekf')
plt.plot(data['t'][ekf_updates], data['psi_opti'][ekf_updates], label='opti')
plt.xlabel('t [s]')
plt.ylabel('psi [rad]')
plt.legend()

# ACC BIAS
plt.sca(axs[3,0])
plt.plot(data['t'], data['ekf_acc_b_x'], label='ekf')
plt.xlabel('t [s]')
plt.ylabel('acc_b_x [m/s^2]')
plt.legend()
plt.sca(axs[3,1])
plt.plot(data['t'], data['ekf_acc_b_y'], label='ekf')
plt.xlabel('t [s]')
plt.ylabel('acc_b_y [m/s^2]')
plt.legend()
plt.sca(axs[3,2])
plt.plot(data['t'], data['ekf_acc_b_z'], label='ekf')
plt.xlabel('t [s]')
plt.ylabel('acc_b_z [m/s^2]')
plt.legend()

# GYRO BIAS
plt.sca(axs[4,0])
plt.plot(data['t'], data['ekf_gyro_b_x'], label='ekf')
plt.xlabel('t [s]')
plt.ylabel('gyro_b_x [rad/s]')
plt.legend()
plt.sca(axs[4,1])
plt.plot(data['t'], data['ekf_gyro_b_y'], label='ekf')
plt.xlabel('t [s]')
plt.ylabel('gyro_b_y [rad/s]')
plt.legend()
plt.sca(axs[4,2])
plt.plot(data['t'], data['ekf_gyro_b_z'], label='ekf')
plt.xlabel('t [s]')
plt.ylabel('gyro_b_z [rad/s]')
plt.legend()

plt.show()

Using matplotlib backend: TkAgg


In [34]:
plt.close('all')
x_res = data['ekf_x'][ekf_updates] - data['x_opti'][ekf_updates]
y_res = data['ekf_y'][ekf_updates] - data['y_opti'][ekf_updates]
z_res = data['ekf_z'][ekf_updates] - data['z_opti'][ekf_updates]
pos_res = np.sqrt(x_res**2 + y_res**2 + z_res**2)
plt.plot(data['t'][ekf_updates], pos_res)

## IMU plots

In [22]:
%matplotlib
plt.close('all')
# plot imu measurements
fig, axs = plt.subplots(3, 2, figsize=(5,5), sharex=True, sharey='col', tight_layout=True)

# ACCELEROMETER
# X
plt.sca(axs[0,0])
plt.plot(data['t'], data['ax'], label='ax')
plt.plot(data['t'], data['ax_filt'], label='ax_filt')
plt.xlabel('t [s]')
plt.ylabel('ax [m/s^2]')
plt.legend()
# Y
plt.sca(axs[1,0])
plt.plot(data['t'], data['ay'], label='ay')
plt.plot(data['t'], data['ay_filt'], label='ay_filt')
plt.xlabel('t [s]')
plt.ylabel('ay [m/s^2]')
plt.legend()
# Z
plt.sca(axs[2,0])
plt.plot(data['t'], data['az'], label='az')
plt.plot(data['t'], data['az_filt'], label='az_filt')
plt.xlabel('t [s]')
plt.ylabel('az [m/s^2]')
plt.legend()

# GYROSCOPE
# X
plt.sca(axs[0,1])
plt.plot(data['t'], data['p'], label='p')
plt.xlabel('t [s]')
plt.ylabel('p [rad/s]')
plt.legend()
# Y
plt.sca(axs[1,1])
plt.plot(data['t'], data['q'], label='q')
plt.xlabel('t [s]')
plt.ylabel('q [rad/s]')
plt.legend()
# Z
plt.sca(axs[2,1])
plt.plot(data['t'], data['r'], label='r')
plt.xlabel('t [s]')
plt.ylabel('r [rad/s]')
plt.legend()

plt.show()

Using matplotlib backend: TkAgg


In [ ]:
keys = [
    't',
    'x_opti', 'y_opti', 'z_opti',
    'phi_opti', 'theta_opti', 'psi_opti',
    'ax', 'ay', 'az',
    'p', 'q', 'r',
    'ekf_x', 'ekf_y', 'ekf_z',
    'ekf_vx', 'ekf_vy', 'ekf_vz',
    'ekf_phi', 'ekf_theta', 'ekf_psi',
    'ekf_acc_b_x', 'ekf_acc_b_y', 'ekf_acc_b_z',
    'ekf_gyro_b_x', 'ekf_gyro_b_y', 'ekf_gyro_b_z'
]
    
ekf_data = {k: data[k] for k in keys}
# save
# np.savez('ekf_data_6may_8ms_delay.npz', **ekf_data)

## Thrust and drag model

In [10]:
nominal_params = {
    'k_x': -6.0e-5,
    'k_y': -6.0e-5,
    'k_w': -2.5e-6,
    # actuator model
    'tau': 0.04,
    'k': 0.98,
    'w_min': 260.34,
    'w_max': 3365.29    
}

def fit_thrust_drag_model(data):
    fig, axs = plt.subplots(1, 3, figsize=(10, 10), sharex=True, sharey=True)
    
    # THRUST MODEL ------------------------------------------------------------------------------
    # Eq. 2 from https://doi.org/10.1016/j.robot.2023.104588
    # az = k_w*sum(omega_i**2) + k_h*(vbx**2+vby**2) + k_z*vbz*sum(omega_i)
    # we will find k_w, k_h, k_z by linear regression
    X = np.stack([
        data['omega[0]']**2 + data['omega[1]']**2 + data['omega[2]']**2 + data['omega[3]']**2,
        # data['vbx']**2 + data['vby']**2,
        # data['vz']*(data['omega[0]']+data['omega[1]']+data['omega[2]']+data['omega[3]'])
    ])
    Y = data['az']
    k_w, = A = np.linalg.lstsq(X.T, Y, rcond=None)[0]
    A_nom = np.array([nominal_params['k_w']])
    
    axs[0].plot(data['t'], Y, label='az', alpha=0.2)
    axs[0].plot(data['t'], data['az_filt'], label='az filt')
    axs[0].plot(data['t'], A@X, label='T model')
    axs[0].plot(data['t'], A_nom@X, label='T model nominal')
    axs[0].set_xlabel('t [s]')
    axs[0].set_ylabel('acc [m/s^2]')
    axs[0].legend()
    axs[0].set_title('Thrust model: \n az = k_w*sum(omega_i**2) \n k_w = {:.2e}'.format(k_w))
    # axs[0].set_title('Thrust model: \n az = k_w*sum(omega_i**2) + k_h*(vbx**2+vby**2) + k_z*vbz*sum(omega_i) \n k_w, k_h, k_z = {:.2e}, {:.2e}, {:.2e}'.format(k_w, k_h, k_z))
    
    # DRAG MODEL X ------------------------------------------------------------------------------
    # Eq. 2 from https://doi.org/10.1016/j.robot.2023.104588
    # ax = -k_x*vbx*sum(omega_i)
    # we will find k_x by linear regression
    X = np.stack([data['vbx']*(data['omega[0]']+data['omega[1]']+data['omega[2]']+data['omega[3]'])])
    # X = np.stack([data['vbx']])
    Y = data['ax']
    k_x, = A = np.linalg.lstsq(X.T, Y, rcond=None)[0]
    A_nom = np.array([nominal_params['k_x']])
    
    axs[1].plot(data['t'], Y, label='ax', alpha=0.2)
    axs[1].plot(data['t'], data['ax_filt'], label='ax filt')
    axs[1].plot(data['t'], A@X, label='Dx model')
    axs[1].plot(data['t'], A_nom@X, label='Dx model nominal')
    axs[1].set_xlabel('t [s]')
    axs[1].set_ylabel('acc [m/s^2]')
    axs[1].legend()
    axs[1].set_title('Drag model X: \n ax = k_x*vbx*sum(omega_i) \n k_x = {:.2e}'.format(k_x))
    
    # DRAG MODEL Y ------------------------------------------------------------------------------
    # Eq. 2 from https://doi.org/10.1016/j.robot.2023.104588
    # ay = -k_y*vby*sum(omega_i)
    # we will find k_y by linear regression
    X = np.stack([data['vby']*(data['omega[0]']+data['omega[1]']+data['omega[2]']+data['omega[3]'])])
    # X = np.stack([data['vby']])
    Y = data['ay']
    k_y, = A = np.linalg.lstsq(X.T, Y, rcond=None)[0]
    A_nom = np.array([nominal_params['k_y']])
    
    axs[2].plot(data['t'], Y, label='ay', alpha=0.2)
    axs[2].plot(data['t'], data['ay_filt'], label='ay filt')
    axs[2].plot(data['t'], A@X, label='Dy model')
    axs[2].plot(data['t'], A_nom@X, label='Dy model nominal')
    axs[2].set_xlabel('t [s]')
    axs[2].set_ylabel('acc [m/s^2]')
    axs[2].legend()
    axs[2].set_title('Drag model Y: \n ay = k_y*vby*sum(omega_i) \n k_y = {:.2e}'.format(k_y))
    
    plt.show()
    return k_w, k_x, k_y
    

plt.close('all')
k_w, k_x, k_y = fit_thrust_drag_model(data)


Intel MKL ERROR: Parameter 4 was incorrect on entry to DGELSD.


LinAlgError: SVD did not converge in Linear Least Squares

In [ ]:
print('k_w={:.2e}, k_x={:.2e}, k_y={:.2e}'.format(k_w, k_x, k_y))

In [11]:
from scipy.optimize import minimize

def fit_actuator_model(data):
    # the steadystate rpm motor response to the motor command u is described by:
    # w_c = (w_max-w_min)*sqrt(k u**2 + (1-k)*u) + w_min
    # the dynamics of the motor is described by:
    # dw/dt = (w_c - w)/tau
    # dw/dt = ((w_max-w_min)*sqrt(k u**2 + (1-k)*u) + w_min - w)*tau_inv
    # we will find w_min, w_max, k, tau_inv by nonlinear optimization
    
    def get_w_est(params, u, w):
        w_min, w_max, k, tau_inv = params
        w_c = (w_max-w_min)*np.sqrt(k*u**2 + (1-k)*u) + w_min
        # progate the dynamics
        w_est = np.zeros_like(u)
        w_est[0] = w[0]
        for i in range(1, len(w_est)):
            dt = data['t'][i] - data['t'][i-1]
            w_est[i] = w_est[i-1] + (w_c[i] - w_est[i-1])*dt*tau_inv
        return w_est

    def get_error(params, u, w):
        return np.linalg.norm(get_w_est(params, u, w) - w)
    
    # w_min, w_max, k, tau_inv
    initial_guess = [285, 2700, 0.75, 100]
    bounds = [(0, 4000), (0, 4000), (0, 1), (1, 1000.)]
    
    # minimize for each motor
    err_1 = lambda x: get_error(x, data['u1'], data['omega[0]'])
    err_2 = lambda x: get_error(x, data['u2'], data['omega[1]'])
    err_3 = lambda x: get_error(x, data['u3'], data['omega[2]'])
    err_4 = lambda x: get_error(x, data['u4'], data['omega[3]'])
    err_tot = lambda x: err_1(x) + err_2(x) + err_3(x) + err_4(x)
    
    print('optimizing...')
    res_1 = minimize(err_1, initial_guess, bounds=bounds)
    res_2 = minimize(err_2, initial_guess, bounds=bounds)
    res_3 = minimize(err_3, initial_guess, bounds=bounds)
    res_4 = minimize(err_4, initial_guess, bounds=bounds)
    res_tot = minimize(err_tot, initial_guess, bounds=bounds)
    # res_nom = [nominal_params['w_min'], nominal_params['w_max'], nominal_params['k'], 1/nominal_params['tau']]
    
    # set k to 45
    # res_tot.x[2] = 0.45
    
    # plot results
    fig, axs = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)
    
    axs[0,0].plot(data['t'], data['omega[0]'], label='w')
    axs[0,0].plot(data['t'], get_w_est(res_1.x, data['u1'], data['omega[0]']), label='w est')
    axs[0,0].plot(data['t'], get_w_est(res_tot.x, data['u1'], data['omega[0]']), label='w est tot')
    # axs[0,0].plot(data['t'], get_w_est(res_nom, data['u1'], data['omega[0]']), label='w est nom')
    axs[0,0].set_xlabel('t [s]')
    axs[0,0].set_ylabel('w [rad/s]')
    axs[0,0].legend()
    axs[0,0].set_title('Motor 1: w_min = {:.2f}, w_max = {:.2f}, k = {:.2f}, 1/tau = {:.2f}'.format(*res_1.x))
    
    axs[0,1].plot(data['t'], data['omega[1]'], label='w')
    axs[0,1].plot(data['t'], get_w_est(res_2.x, data['u2'], data['omega[1]']), label='w_est')
    axs[0,1].plot(data['t'], get_w_est(res_tot.x, data['u2'], data['omega[1]']), label='w est tot')
    # axs[0,1].plot(data['t'], get_w_est(res_nom, data['u2'], data['omega[1]']), label='w est nom')
    axs[0,1].set_xlabel('t [s]')
    axs[0,1].set_ylabel('w [rad/s]')
    axs[0,1].legend()
    axs[0,1].set_title('Motor 2: w_min = {:.2f}, w_max = {:.2f}, k = {:.2f}, 1/tau = {:.2f}'.format(*res_2.x))
    
    axs[1,0].plot(data['t'], data['omega[2]'], label='w')
    axs[1,0].plot(data['t'], get_w_est(res_3.x, data['u3'], data['omega[2]']), label='w_est')
    axs[1,0].plot(data['t'], get_w_est(res_tot.x, data['u3'], data['omega[2]']), label='w est tot')
    # axs[1,0].plot(data['t'], get_w_est(res_nom, data['u3'], data['omega[2]']), label='w est nom')
    axs[1,0].set_xlabel('t [s]')
    axs[1,0].set_ylabel('w [rad/s]')
    axs[1,0].legend()
    axs[1,0].set_title('Motor 3: w_min = {:.2f}, w_max = {:.2f}, k = {:.2f}, 1/tau = {:.2f}'.format(*res_3.x))
    
    axs[1,1].plot(data['t'], data['omega[3]'], label='w')
    axs[1,1].plot(data['t'], get_w_est(res_4.x, data['u4'], data['omega[3]']), label='w_est')
    axs[1,1].plot(data['t'], get_w_est(res_tot.x, data['u4'], data['omega[3]']), label='w est tot')
    # axs[1,1].plot(data['t'], get_w_est(res_nom, data['u4'], data['omega[3]']), label='w est nom')
    axs[1,1].set_xlabel('t [s]')
    axs[1,1].set_ylabel('w [rad/s]')
    axs[1,1].legend()
    axs[1,1].set_title('Motor 4: w_min = {:.2f}, w_max = {:.2f}, k = {:.2f}, 1/tau = {:.2f}'.format(*res_4.x))
    
    # suptitle
    fig.suptitle('Actuator model: \n dw/dt = dw/dt = ((w_max-w_min)*sqrt(k u**2 + (1-k)*u) + w_min - w)/tau \n Total fit: w_min = {:.2f}, w_max = {:.2f}, k = {:.2f}, 1/tau = {:.2f}'.format(*res_tot.x))
    
    plt.show()
    return res_tot.x

w_min, w_max, k, tau_inv = fit_actuator_model(data)

optimizing...


In [12]:
print('w_min={:.2f}, w_max={:.2f}, k={:.2f}, tau={:.2f}'.format(w_min, w_max, k, 1/tau_inv))

w_min=261.64, w_max=2742.27, k=0.80, tau=0.04


In [ ]:
Tmax = k_w * (w_max)**2
print('Tmax', Tmax, '[m/s^2] =', Tmax/9.81, 'g')
print('Tmax_total', 4*Tmax, '[m/s^2] =', 4*Tmax/9.81, 'g')

## Moments model

In [20]:
def fit_moments_model(data):
    # model from https://doi.org/10.1016/j.robot.2023.104588
    # d_p     = (q*r*(Iyy-Izz) + Mx)/Ixx = Jx*q*r + Mx_
    # d_q     = (p*r*(Izz-Ixx) + My)/Iyy = Jy*p*r + My_
    # d_r     = (p*q*(Ixx-Iyy) + Mz)/Izz = Jz*p*q + Mz_
    
    # where    
    # Mx_ = k_p1*omega_1**2 + k_p2*omega_2**2 + k_p3*omega_3**2 + k_p4*omega_4**2
    # My_ = k_q1*omega_1**2 + k_q2*omega_2**2 + k_q3*omega_3**2 + k_q4*omega_4**2
    # Mz_ = k_r1*omega_1 + k_r2*omega_2 + k_r3*omega_3 + k_r4*omega_4 + k_r5*d_omega_1 + k_r6*d_omega_2 + k_r7*d_omega_3 + k_r8*d_omega_4
    
    # to get the derivative we use a low pass filter
    cutoff = 64 # Hz
    sos = sp.signal.butter(2, cutoff, 'low', fs=1/np.mean(np.diff(data['t'])), output='sos')
    
    dp = sp.signal.sosfiltfilt(sos, np.gradient(data['p'])/np.gradient(data['t']))
    dq = sp.signal.sosfiltfilt(sos, np.gradient(data['q'])/np.gradient(data['t']))
    dr = sp.signal.sosfiltfilt(sos, np.gradient(data['r'])/np.gradient(data['t']))
    
    domega_1 = sp.signal.sosfiltfilt(sos, np.gradient(data['omega[0]'])/np.gradient(data['t']))
    domega_2 = sp.signal.sosfiltfilt(sos, np.gradient(data['omega[1]'])/np.gradient(data['t']))
    domega_3 = sp.signal.sosfiltfilt(sos, np.gradient(data['omega[2]'])/np.gradient(data['t']))
    domega_4 = sp.signal.sosfiltfilt(sos, np.gradient(data['omega[3]'])/np.gradient(data['t']))
    
    params = np.load('params/aggressive_cmds2.npz')
    
    fig, axs = plt.subplots(3, 2, figsize=(10, 10), sharex=True, sharey=True)
    
    X = np.stack([
        data['omega[0]']**2,
        data['omega[1]']**2,
        data['omega[2]']**2,
        data['omega[3]']**2,
    ])
    Y = dp
    A = np.linalg.lstsq(X.T, Y, rcond=None)[0]
    k_p1, k_p2, k_p3, k_p4 = A
    dp_fit = A@X
    
    axs[0,0].plot(data['t'], Y, label='dp')
    axs[0,0].plot(data['t'], dp_fit, label='dp fit')
    axs[0,0].set_xlabel('t [s]')
    axs[0,0].set_ylabel('dp [rad/s^2]')
    axs[0,0].legend()
    axs[0,0].set_title('dp = k_p1*w1**2 + k_p2*w2**2 + k_p3*w3**2 + k_p4*w4**2 \n k_p1, k_p2, k_p3, k_p4 = {:.2e}, {:.2e}, {:.2e}, {:.2e}'.format(*A))
    
    X = np.stack([
        data['omega[0]']**2,
        data['omega[1]']**2,
        data['omega[2]']**2,
        data['omega[3]']**2,
    ])
    Y = dq
    A = np.linalg.lstsq(X.T, Y, rcond=None)[0]
    k_q1, k_q2, k_q3, k_q4 = A
    dq_fit = A@X
    
    axs[1,0].plot(data['t'], Y, label='dq')
    axs[1,0].plot(data['t'], dq_fit, label='dq fit')
    axs[1,0].set_xlabel('t [s]')
    axs[1,0].set_ylabel('dq [rad/s^2]')
    axs[1,0].legend()
    axs[1,0].set_title('dq = k_q1*w1**2 + k_q2*w2**2 + k_q3*w3**2 + k_q4*w4**2 \n k_q1, k_q2, k_q3, k_q4 = {:.2e}, {:.2e}, {:.2e}, {:.2e}'.format(*A))
    
    
    X = np.stack([
        data['omega[0]'],
        data['omega[1]'],
        data['omega[2]'],
        data['omega[3]'],
        domega_1,
        domega_2,
        domega_3,
        domega_4,
    ])
    Y = dr
    A = np.linalg.lstsq(X.T, Y, rcond=None)[0]
    k_r1, k_r2, k_r3, k_r4, k_r5, k_r6, k_r7, k_r8 = A
    dr_fit = A@X

     
    axs[2,0].plot(data['t'], Y, label='dr')
    axs[2,0].plot(data['t'], dr_fit, label='dr fit')
    axs[2,0].set_xlabel('t [s]')
    axs[2,0].set_ylabel('dr [rad/s^2]')
    axs[2,0].legend()
    axs[2,0].set_title('dr = k_r1*w1 + k_r2*w2 + k_r3*w3 + k_r4*w4 + k_r5*dw1 + k_r6*dw2 + k_r7*dw3 + k_r8*dw4 \n k_r1, k_r2, k_r3, k_r4, k_r5, k_r6, k_r7, k_r8 = {:.2e}, {:.2e}, {:.2e}, {:.2e}, {:.2e}, {:.2e}, {:.2e}, {:.2e}'.format(*A))
    
    # 3 plots with p,q,r
    axs[0,1].plot(data['t'], dp_fit-dp, label='dp fit error')
    axs[0,1].set_xlabel('t [s]')
    axs[0,1].set_ylabel('p [rad/s]')
    axs[0,1].legend()
    
    axs[1,1].plot(data['t'], dq_fit-dq, label='dq fit error')
    axs[1,1].set_xlabel('t [s]')
    axs[1,1].set_ylabel('q [rad/s]')
    axs[1,1].legend()
    
    axs[2,1].plot(data['t'], dr_fit-dr, label='dr fit error')
    axs[2,1].set_xlabel('t [s]')
    axs[2,1].set_ylabel('r [rad/s]')
    axs[2,1].legend()
    
    plt.show()
    return k_p1, k_p2, k_p3, k_p4, k_q1, k_q2, k_q3, k_q4, k_r1, k_r2, k_r3, k_r4, k_r5, k_r6, k_r7, k_r8

%matplotlib
plt.close('all')
k_p1, k_p2, k_p3, k_p4, k_q1, k_q2, k_q3, k_q4, k_r1, k_r2, k_r3, k_r4, k_r5, k_r6, k_r7, k_r8 = fit_moments_model(data)

Using matplotlib backend: TkAgg


In [ ]:
plt.plot((-data['omega[0]']+data['omega[1]']+data['omega[2]']-data['omega[3]'])*1.4e-2)

In [ ]:
print('Roll moments')
print('k_p1={:.2e}, k_p2={:.2e}, k_p3={:.2e}, k_p4={:.2e}'.format(k_p1, k_p2, k_p3, k_p4))
print('Pitch moments')
print('k_q1={:.2e}, k_q2={:.2e}, k_q3={:.2e}, k_q4={:.2e}'.format(k_q1, k_q2, k_q3, k_q4))
print('Yaw moments')
print('k_r1={:.2e}, k_r2={:.2e}, k_r3={:.2e}, k_r4={:.2e}'.format(k_r1, k_r2, k_r3, k_r4))
print('k_r5={:.2e}, k_r6={:.2e}, k_r7={:.2e}, k_r8={:.2e}'.format(k_r5, k_r6, k_r7, k_r8))

## INDI Model

In [ ]:
from scipy.optimize import minimize

def fit_rate_INDI_model(data):
    # the rate is a first order system of the rate command
    
    def get_1st_order_est(tau, u, x):
        # d/dt x = (u - x)/tau
        x_est = np.zeros_like(u)
        x_est[0] = x[0]
        for i in range(1, len(x_est)):
            dt = data['t'][i] - data['t'][i-1]
            x_est[i] = x_est[i-1] + (u[i] - x_est[i-1])*dt/tau
        return x_est

    def get_error(tau, u, x):
        return np.linalg.norm(get_1st_order_est(tau, u, x) - x)
    
    err_p = lambda x: get_error(x, data['p_sp'], data['p'])
    err_q = lambda x: get_error(x, data['q_sp'], data['q'])
    err_r = lambda x: get_error(x, data['r_sp'], data['r'])
    
    res_p = minimize(err_p, x0=0.1, bounds=[(0.01, 1)])
    res_q = minimize(err_q, x0=0.1, bounds=[(0.01, 1)])
    res_r = minimize(err_r, x0=0.1, bounds=[(0.01, 1)])
    
    # plot results
    fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
    axs[0].plot(data['t'], data['p_sp'], label='p_sp')
    axs[0].plot(data['t'], data['p'], label='p')
    axs[0].plot(data['t'], get_1st_order_est(res_p.x, data['p_sp'], data['p']), label='p_est')
    axs[0].set_xlabel('t [s]')
    axs[0].set_ylabel('p [rad/s]')
    axs[0].legend()
    axs[0].set_title('d/dt p_est = (p_sp - p)/tau \n tau = {:.2f}'.format(res_p.x[0]))
    
    axs[1].plot(data['t'], data['q_sp'], label='q_sp')
    axs[1].plot(data['t'], data['q'], label='q')
    axs[1].plot(data['t'], get_1st_order_est(res_q.x, data['q_sp'], data['q']), label='q_est')
    axs[1].set_xlabel('t [s]')
    axs[1].set_ylabel('q [rad/s]')
    axs[1].legend()
    axs[1].set_title('d/dt q_est = (q_sp - q)/tau \n tau = {:.2f}'.format(res_q.x[0]))
    
    
    axs[2].plot(data['t'], data['r_sp'], label='r_sp')
    axs[2].plot(data['t'], data['r'], label='r')
    axs[2].plot(data['t'], get_1st_order_est(res_r.x, data['r_sp'], data['r']), label='r_est')
    axs[2].set_xlabel('t [s]')
    axs[2].set_ylabel('r [rad/s]')
    axs[2].legend()
    axs[2].set_title('d/dt r_est = (r_sp - r)/tau \n tau = {:.2f}'.format(res_r.x[0]))
    
    plt.show()
    return res_p.x, res_q.x, res_r.x

tau_p, tau_q, tau_r = fit_rate_INDI_model(data)    

# Save model parameters

In [ ]:
import os
# put all params in a dict
params = {
    'k_w': k_w,
    'k_x': k_x,
    'k_y': k_y,
    'omega_0': omega_0,
    'omega_m': omega_m,
    'k': k,
    'tau_inv': tau_inv,
    'k_p': k_p,
    'k_q1': k_q1,
    'k_q2': k_q2,
    'k_r': k_r,
    'k_r2': k_r2
}

# save into npz file (in folder named params)
if os.path.exists('params') == False:
    os.mkdir('params')
    
name = path.split('/')[-1].split('.')[0]
np.savez('params/'+name+'.npz', **params)

In [ ]:
# !!!!!!!!!!! NOT WORKING YET !!!!!!!!!!!!!!!
# INDI PARAMETER ESTIMATION
# pseudocontrol: v = [T, dp, dq, dr]
# control: u = [u1, u2, u3, u4]
# dv = G*du
# we will find the G matrix by linear regression
u = np.stack([
    data['u[0]'],
    data['u[1]'],
    data['u[2]'],
    data['u[3]'],
    
])
v = np.stack([
    data['az_filt'],
    data['alpha[0]'],
    data['alpha[1]'],
    data['alpha[2]'],
])

du = np.gradient(u, axis=1)/np.gradient(data['t'])
dv = np.gradient(v, axis=1)/np.gradient(data['t'])

# time interval
du_ = du#[:, 500*0:500*12]
dv_ = dv#[:, 500*0:500*12]

G = np.linalg.lstsq(du_.T, dv_.T, rcond=None)[0]

# pretty print G matrix
print('G = ')
for i in range(4):
    print('[' + ', '.join(['{:.2f}'.format(G[i,j]) for j in range(4)]) + ']')
    


# plot results
plt.close('all')
fig, axs = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)

dv_est = G@du

axs[0,0].plot(data['t'], dv_est[0], label='delta T model')
axs[0,0].plot(data['t'], dv[0], label='delta T')
axs[0,0].set_xlabel('t [s]')
axs[0,0].set_ylabel('dv [m/s^2]')
axs[0,0].legend()

axs[0,1].plot(data['t'], dv[1], label='delta alpha[0]')
axs[0,1].plot(data['t'], dv_est[1], label='delta alpha[0] model')
axs[0,1].set_xlabel('t [s]')
axs[0,1].set_ylabel('dv [rad/s^2]')
axs[0,1].legend()

axs[1,0].plot(data['t'], dv[2], label='delta alpha[1]')
axs[1,0].plot(data['t'], dv_est[2], label='delta alpha[1] model')
axs[1,0].set_xlabel('t [s]')
axs[1,0].set_ylabel('dv [rad/s^2]')
axs[1,0].legend()

axs[1,1].plot(data['t'], dv[3], label='delta alpha[1]')
axs[1,1].plot(data['t'], dv_est[3], label='delta alpha[1] model')
axs[1,1].set_xlabel('t [s]')
axs[1,1].set_ylabel('dv [rad/s^2]')
axs[1,1].legend()

plt.show()